# Tutorial 3: Audio Generation with Room Accoustic

<p align="right" style="margin-right: 8px;">
    <a target="_blank" href="https://colab.research.google.com/github/idiap/sdialog/blob/main/tutorials/01_audio/3.accoustic_simulation-customer_service.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p>

## Getting started

### Environment Setup

Let's first check if our environment is all set up.

In [ ]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
import os
from IPython import get_ipython
from IPython.display import Audio, display

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    !sudo apt-get install sox ffmpeg
    !sudo apt-get -qq -y install espeak-ng > /dev/null 2>&1
    %pip install -q kokoro>=0.9.4
    # Installing sdialog
    !git clone https://github.com/idiap/sdialog.git
    %cd sdialog
    %pip install -e .[audio]
    %cd ..
else:
    print("Running in Jupyter Notebook")

### Load the example dialogue

In order to run the next steps in a fast manner, we will start from an existing dialog generated using previous tutorials:

In [ ]:
from sdialog import Dialog

path_dialog = "../../tests/data/customer_support_dialogue.json"

if not os.path.exists(path_dialog):
    !wget https://raw.githubusercontent.com/idiap/sdialog/refs/heads/main/tests/data/customer_support_dialogue.json
    path_dialog = "./customer_support_dialogue.json"

original_dialog = Dialog.from_file(path_dialog)
original_dialog.print()

# Tutorial

### Instanciate voices database

In [ ]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
kokoro_voice_database = HuggingfaceVoiceDatabase("sdialog/voices-kokoro")

### Instanciate TTS model

In [ ]:
from sdialog.audio.tts import KokoroTTS
tts_engine = KokoroTTS()

## Setup stage: Audio Dialog and Audio Pipeline

In [ ]:
from sdialog.audio.dialog import AudioDialog
from sdialog.audio.pipeline import AudioPipeline

Convert the original dialog into a audio enhanced dialog

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

Instanciate the audio pipeline in order to use `Kokoro` (`tts_engine`) as the TTS model and save the audios outputs of all the dialogs into the directory `./audio_outputs`.

The voices are sampled from the `kokoro_voice_database` based on the persona attributes `age`, `gender` and `language`, as assigned during the original textual dialog.

In [ ]:
os.makedirs("./audio_outputs_customer_support", exist_ok=True)
audio_pipeline = AudioPipeline(
    voice_database=kokoro_voice_database,
    tts_engine=tts_engine,
    dscaper_data_path="./dscaper_data_customer_support",
    dir_audio="./audio_outputs_customer_support",
)

In [ ]:
# Populate the sound events database (for example the background sound of the customer support room with phone ringing, people talking, ...)
# audio_pipeline.populate_dscaper(["<username>/<the_name_of_the_dataset>"])

or if you encounter any issue during the download due to timeout:

In [ ]:
%%script false --no-raise-error
!hf download sdialog/background --repo-type dataset
!hf download sdialog/foreground --repo-type dataset

Now let's generate a medical room it will be enough and display it's shape and content:

In [ ]:
from sdialog.audio.utils import SourceVolume, SourceType
from sdialog.audio.jsalt import MedicalRoomGenerator, RoomRole

In [ ]:
room = MedicalRoomGenerator().generate(args={"room_type": RoomRole.EXAMINATION})
img = room.to_image()
display(img)
img.save("room.png")

We can also optionaly position the speakers 1 and 2 into the room by assigning them places around available furnitures in the room or to a 3D position in the environment.

By default the speakers are positionned around the center of the room.

Here the `MedicalRoomGenerator` is generating rooms with a predefined list of furnitures (desk, sink, door, ...) that can be used:

In [ ]:
from sdialog.audio.room import SpeakerSide, Role, RoomPosition

In [ ]:
room.place_speaker_around_furniture(speaker_name=Role.SPEAKER_1, furniture_name="desk", max_distance=1.0, side=SpeakerSide.FRONT)
room.place_speaker_around_furniture(speaker_name=Role.SPEAKER_2, furniture_name="desk", max_distance=1.0, side=SpeakerSide.BACK)

You can see the new positions of the speakers:

In [ ]:
img = room.to_image()
display(img)

Perform the inference of the audio pipeline on the previously converted dialog. In this case we will focus on generating the "unprocessed" audio, which consist of the agregation of all utterances from the dialog. Rather than using the dialog identifier as the name of the directory, we are using here a custom directory name `demo_dialog_room_accoustic` which will be saved at `./audio_outputs_customer_support/demo_dialog_room_accoustic/`. 

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    perform_room_acoustics=True,
    environment={
        "room": room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
        "background_effect": "white_noise",
        "foreground_effect": "ac_noise_minimal",
        "foreround_effect_position": RoomPosition.TOP_RIGHT,
        "source_volumes": {
            SourceType.ROOM: SourceVolume.HIGH,
            SourceType.BACKGROUND: SourceVolume.VERY_LOW
        },
        "kwargs_pyroom": {
            "ray_tracing": True,
            "air_absorption": True
        }
    },
    dialog_dir_name="demo_dialog_room_accoustic",
    room_name="my_room_config_1"
)

In [ ]:
dialog.display()